# Manually calculate Fractional Crystalization using Equilibrate

In [ ]:
from thermoengine import equilibrate, model, core, phases
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
%matplotlib inline

In [ ]:
import magmaforge

In [ ]:
from magmaforge import system

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
src_obj = core.get_src_object('EquilibrateUsingMELTSv102')
modelDB = model.Database(liq_mod='v1.0')

In [ ]:
Liquid = modelDB.get_phase('Liq')
Feldspar = modelDB.get_phase('Fsp')
Quartz = modelDB.get_phase('Qz')
Spinel = modelDB.get_phase('SplS')
Opx = modelDB.get_phase('Opx')
RhomOx = modelDB.get_phase('Rhom')

The Berman model database provides the SWIM water model by default.  Instead, override that choice by instantiating the MELTS 1.0.2 water model directly.

In [ ]:
Water = phases.PurePhase('WaterMelts', 'H2O', calib=False)

## Define elements in system and phases in system

In [ ]:
elm_sys = ['H','O','Na','Mg','Al','Si','P','K','Ca','Ti','Cr','Mn','Fe','Co','Ni']
phs_sys = [Liquid, Feldspar, Water, Quartz, Spinel, Opx, RhomOx]

## Composition of the system
This is a high-silica rhyolite

In [ ]:
grm_oxides = {
    'SiO2':  77.5, 
    'TiO2':   0.08, 
    'Al2O3': 12.5, 
    'Fe2O3':  0.207,
    'Cr2O3':  0.0, 
    'FeO':    0.473, 
    'MnO':    0.0,
    'MgO':    0.03, 
    'NiO':    0.0, 
    'CoO':    0.0,
    'CaO':    0.43, 
    'Na2O':   3.98, 
    'K2O':    4.88, 
    'P2O5':   0.0, 
    'H2O':    5.5
}

Cast this composition as moles of elements for input to the Equilibrate class

In [ ]:
mol_oxides = core.chem.format_mol_oxide_comp(grm_oxides, convert_grams_to_moles=True)
moles_end,oxide_res = Liquid.calc_endmember_comp(
    mol_oxide_comp=mol_oxides, method='intrinsic', output_residual=True)
if not Liquid.test_endmember_comp(moles_end):
    print ("Calculated composition is infeasible!")
mol_elm = Liquid.covert_endmember_comp(moles_end,output='moles_elements')

In [ ]:
mol_oxides

In [ ]:
blk_cmp = []
for elm in elm_sys:
    index = core.chem.PERIODIC_ORDER.tolist().index(elm)
    blk_cmp.append(mol_elm[index])
blk_cmp = np.array(blk_cmp)

## Instantiate class instance and run calculation

In [ ]:
equil = equilibrate.Equilibrate(elm_sys, phs_sys)

In [ ]:
t = 1050.0
p = 1750.0
state0 = equil.execute(t, p, bulk_comp=blk_cmp, debug=0, stats=True)

liq_wtcomp = state.compositions(phase_name='Liquid', ctype='oxides', units='wt%')
print(liq_wtcomp)


In [ ]:
import copy
statecp = copy.copy(state0)
statecp.temperature

In [ ]:
state1 = equil.execute(t-100, p, state=state0, debug=0, stats=True)

In [ ]:
state1.tot_grams_phase('Liquid')

In [ ]:
statecp.tot_grams_phase('Liquid')

In [ ]:
# OXIDES = [ox for ox in core.chem.OXIDE_ORDER if ox != 'CO2']
OXIDES = core.chem.OXIDE_ORDER

In [ ]:
pd.Series(liq_wtcomp, index=OXIDES)

In [ ]:
state.print_state()

## explore state output variables and methods

In [ ]:
liq_props = state.phase_d['Liquid']
liq_props.keys()

In [ ]:
liq_props['moles']

In [ ]:
liq_props['end_molwts']

In [ ]:
state.tot_moles_elements()

In [ ]:

state.compositions(phase_name='Liquid')
state.compositions?

In [ ]:

state.compositions(phase_name='Liquid', ctype='oxides', units='wt%')

In [ ]:
state.phase_d.keys()

In [ ]:
state.tot_moles_elements()